In [1]:
from keras.preprocessing import sequence
from keras.datasets import imdb
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Layer, Lambda, Input, Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import tensorflow as tf

import tensorflow_hub as hub


In [2]:

# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np

In [3]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history[metric])
  plt.plot(history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [4]:
vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('Loaded dataset with {} training samples, {} test samples'
      .format(len(X_train), len(X_test)))


Loaded dataset with 25000 training samples, 25000 test samples


In [5]:
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}

In [6]:
print('Maximum review length: {}'.format(
len(max((X_train + X_test), key=len))))

Maximum review length: 2697


In [7]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [14]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    #def compute_mask(self, inputs, mask=None):
        #return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [9]:
# Function to build model
def build_model(): 
    input_text = layers.Input(shape=(1,), dtype="string")
    embedding = ElmoEmbeddingLayer()(input_text)
    dense = layers.Dense(256, activation='relu')(embedding)
    pred = layers.Dense(1, activation='sigmoid')(dense)

    model = Model(inputs=[input_text], outputs=pred)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [35]:
#elmo_module = hub.load("https://tfhub.dev/google/elmo/3")
elmo_module = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
#elmo_module.signatures['default'](tf.squeeze(tf.cast(x, tf.string))

def ELMoEmbedding2(x):
    #return elmo_module.signatures['default'](tf.squeeze(tf.cast(x, tf.string)))
    return elmo_module(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]


#input_comment = Input(shape=(1,), dtype="string", name='input_comment')
    #embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_comment)

RuntimeError: Exporting/importing meta graphs is not supported when eager execution is enabled. No graph exists when eager execution is enabled.

In [ ]:
embedding_size=32
input_text = Input(shape=(1,), dtype=tf.string)

embedding = Lambda(ELMoEmbedding2, output_shape=(1024, ))(input_text)

In [27]:
embedding_size=32
input_text = Input(shape=(1,), dtype=tf.string)
#embedding_size=32

#embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)


model=Sequential()
#model.add(elmo_module)
model.add(Lambda(ELMoEmbedding2, output_shape=(1024, ))(input_text))
model.add(Bidirectional(LSTM(100,  return_sequences=True)))
model.add(Bidirectional(LSTM(50,  return_sequences=True)))
model.add(Dense(1, activation='sigmoid'))

model.build()

print(model.summary())

TypeError: 'AutoTrackable' object is not callable

In [23]:
embedding_size=32
input_text = Input(shape=(1,), dtype=tf.string)

model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(ELMoEmbedding2)(input_text)
#model.add(elmo)(inputs=[tokens, seq_lens], outputs=out)
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

TypeError: The added layer must be an instance of class Layer. Found: <function ELMoEmbedding2 at 0x7fb61821a790>

In [34]:
embed = hub.load("https://tfhub.dev/google/elmo/3")

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),
    signature="default", as_dict=True)["default"]

# export PYTHONWARNINGS="ignore"
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
#pred = Dense(2, activation='softmax')(dense)
#model = Model(inputs=[input_text], outputs=pred)
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

TypeError: 'AutoTrackable' object is not callable

In [28]:
# Build and fit
input_text = layers.Input(shape=(1,), dtype="string")
embedding = ElmoEmbeddingLayer()(input_text)
#dense = layers.Dense(256, activation='relu')(embedding)
#pred = layers.Dense(1, activation='sigmoid')(dense)
#
#model = Model(inputs=[input_text], outputs=pred)
#
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.summary()
#
#model.compile(loss='binary_crossentropy', 
#             optimizer='adam', 
#             metrics=['accuracy'])

KeyboardInterrupt: 

In [ ]:
# save best parameter estimate using val loss.
checkpointer = ModelCheckpoint("best_base_model.tf", monitor="val_loss", mode="auto", save_best_only = True, verbose=1)
# early stopping.
earlystoper = EarlyStopping(monitor = 'val_loss', patience = 2, verbose = 1) # patience is num. epochs wo improvement.

# save the training history.
csv_logger = CSVLogger('trainingBaseModel.log', separator=',', append=False) 
# compile the callbacks together
callbackser = [earlystoper, checkpointer, csv_logger]

In [ ]:
batch_size = 64
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), 
          batch_size=batch_size, epochs=num_epochs)

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

In [ ]:
history = pd.read_csv('trainingBaseModel.log', sep=',', engine='python')